# Advent of Code #


In [1]:
# set up the environment
import numpy as np

## Day 4: Giant Squid ##

### Part 1 ###
You're already almost 1.5km (almost a mile) below the surface of the ocean, already so deep that you can't see any sunlight. What you can see, however, is a giant squid that has attached itself to the outside of your submarine.

Maybe it wants to play bingo?

Bingo is played on a set of boards each consisting of a 5x5 grid of numbers. Numbers are chosen at random, and the chosen number is marked on all boards on which it appears. (Numbers may not appear on all boards.) If all numbers in any row or any column of a board are marked, that board wins. (Diagonals don't count.)

The submarine has a bingo subsystem to help passengers (currently, you and the giant squid) pass the time. It automatically generates a random order in which to draw numbers and a random set of boards (your puzzle input). For example:

`7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1`

```
22 13 17 11  0
 8  2 23  4 24
21  9 14 16  7
 6 10  3 18  5
 1 12 20 15 19

 3 15  0  2 22
 9 18 13 17  5
19  8  7 25 23
20 11 10 24  4
14 21 16 12  6

14 21 17 24  4
10 16 15  9 19
18  8 23 26 20
22 11 13  6  5
 2  0 12  3  7
```
After the first five numbers are drawn `(7, 4, 9, 5, and 11)`, there are no winners, but the boards are marked as follows (shown here adjacent to each other to save space):
```
22 13 17 11  0         3 15  0  2 22        14 21 17 24  4
 8  2 23  4 24         9 18 13 17  5        10 16 15  9 19
21  9 14 16  7        19  8  7 25 23        18  8 23 26 20
 6 10  3 18  5        20 11 10 24  4        22 11 13  6  5
 1 12 20 15 19        14 21 16 12  6         2  0 12  3  7
```
After the next six numbers are drawn `(17, 23, 2, 0, 14, and 21)`, there are still no winners:
```
22 13 17 11  0         3 15  0  2 22        14 21 17 24  4
 8  2 23  4 24         9 18 13 17  5        10 16 15  9 19
21  9 14 16  7        19  8  7 25 23        18  8 23 26 20
 6 10  3 18  5        20 11 10 24  4        22 11 13  6  5
 1 12 20 15 19        14 21 16 12  6         2  0 12  3  7
```
Finally, `24` is drawn:
```
22 13 17 11  0         3 15  0  2 22        14 21 17 24  4
 8  2 23  4 24         9 18 13 17  5        10 16 15  9 19
21  9 14 16  7        19  8  7 25 23        18  8 23 26 20
 6 10  3 18  5        20 11 10 24  4        22 11 13  6  5
 1 12 20 15 19        14 21 16 12  6         2  0 12  3  7
```
At this point, the third board wins because it has at least one complete row or column of marked numbers (in this case, the entire top row is marked: `14 21 17 24 4`).

The score of the winning board can now be calculated. Start by finding the sum of all unmarked numbers on that board; in this case, the sum is 188. Then, multiply that sum by the number that was just called when the board won, 24, to get the final score, 188 * 24 = 4512.

To guarantee victory against the giant squid, figure out which board will win first. What will your final score be if you choose that board?

### Inport the data and define a bingo card ###

In [2]:
# Import the data into array 
bingo_calls = np.genfromtxt("data/bingo_calls.dat", dtype="i4", delimiter=',')
raw_cards = np.genfromtxt("data/bingo_cards.dat", dtype="i4").reshape(100,5,5)

# define a bingo_card
class BingoCard():
    # A bingo card consists of a 6 x 6 matrix
    # rows 1 - 5 and columns 1 - 5 contain the cards numbers
    # row 6 column 1 - 5 contains the count of the numbers called in that column
    # column 6  rows 1 -5 contains the count of the numbers called in that row
    # row 6 column 6 indicates a winning card - if any of row 6 or column 6 reaches 5

    def __init__(self) -> None:
        self.card = np.zeros((6,6), dtype="i4")
        self.totals = 0
        self.card_no = 0
        self.last_call = -1
        self.__has_won = False
        self.no_of_checks = 0
        

    def __str__(self) -> str:
        card = f'{self.card[:]}'
        return card

    def build(self, lines:np.ndarray, card_no: int) -> None:
        self.totals = np.sum(lines)
        self.card = np.column_stack((lines, [5, 5, 5, 5, 5]))
        self.card = np.row_stack((self.card, [5, 5, 5, 5, 5, 0]))
        self.card_no = card_no

    def check_number(self, number) -> bool:
        if self.__has_won:
            return True
        self.no_of_checks += 1
        res = np.where(self.card[:5, :5] == number)
        if res[0].size == 0:
            return False
        self.last_call = number
        row = res[0]
        col = res[1]
        self.card[row,5] -= 1
        self.card[5,col] -= 1
        self.card[row, col] *= -1
        self. totals -= number
        if self.card[row,5] == 0 or self.card[5, col] == 0:
            self.__has_won = True
            return True

        return False

    def unchecked_sum(self) -> int:
        return self.totals

    def is_winner(self) -> bool:
        return self.__has_won
    


### Set up the board ##

In [3]:
def set_bingo_cards() -> np.ndarray:
    cards = []
    for x, raw_card in enumerate(raw_cards):
        card = BingoCard()
        card.build(raw_card, x+1)
        cards.append(card)
    return cards



### Check for the first winning card ###

In [4]:
bingo_cards = set_bingo_cards()

found_winner = False
winning_card = None
last_call = -1
for call in bingo_calls:
    last_call = call
    for card in bingo_cards:
        if card.check_number(call):
            found_winner = True
            winning_card = card
            break
    if found_winner:
        break

if winning_card :
    unchecked_score = winning_card.unchecked_sum()
    print('Card {} is a winner!\n{}'.format(winning_card.card_no, winning_card))
    print('Last call was {} and the total of uncheched numbers is {}'.format(
        winning_card.last_call, unchecked_score
    ))
    print('Final Score: {}'.format(unchecked_score * last_call))


Card 50 is a winner!
[[-15  56 -39  57  40   3]
 [-67 -59  26  30  90   3]
 [-84  -2 -41 -25  -7   0]
 [ 96 -23  79  99  85   4]
 [ 13  10  86  51  53   5]
 [  2   2   3   4   4   0]]
Last call was 41 and the total of uncheched numbers is 871
Final Score: 35711


### Reset the board and find the last winning card ###

This time play all the numbers in the call list.  At the end we check each ***winning*** card and the one with the highest number of checks before winning is the one that is required. 

In [5]:
bingo_cards = set_bingo_cards()

last_call_ordinal = -1
bingo_card = None

for call in bingo_calls:
    for card in bingo_cards:
        card.check_number(call)

winning_card = None
max_calls = 0
for card in bingo_cards:
    if card.is_winner():
        card_checks = card.no_of_checks
        if card_checks > max_calls:
            max_calls = card_checks
            winning_card = card
    

if winning_card:
    unchecked_score = winning_card.unchecked_sum()
    last_call = winning_card.last_call
    print('Card {}\n{}\nis the last winner!'.format(winning_card.card_no, winning_card))
    print('Last call was {} and the total of uncheched numbers is {}'.format(
        last_call, unchecked_score
    ))
    print('Final Score: {}'.format(unchecked_score * last_call))

Card 31
[[  9 -93 -65 -94 -29   1]
 [ -2  80  -7  16 -79   2]
 [-11  -5 -21  73 -50   1]
 [-20 -70 -37 -48  85   1]
 [-99   3 -55 -58  -8   1]
 [  1   2   0   2   1   0]]
is the last winner!
Last call was 21 and the total of uncheched numbers is 266
Final Score: 5586
